In [ ]:
import numpy as np
import pandas as pd
import re
#from clean_text import *

from tqdm.notebook import tqdm
tqdm.pandas()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rcParams

rcParams['figure.dpi'] = 500
rcParams['savefig.dpi'] = 500
rcParams['font.family'] = 'Arial'
rcParams['axes.labelsize'] = 14
rcParams['axes.titlesize'] = 14
rcParams['xtick.labelsize'] = 14
rcParams['ytick.labelsize'] = 14
rcParams['legend.fontsize'] = 14
rcParams['figure.titlesize'] = 16

In [ ]:
df1 = pd.read_csv('russia_after_48hrs - Correct.csv', encoding = 'utf8')
df2 = pd.read_csv('russia_before_48hrs - Correct.csv', encoding = 'latin1')
df = pd.concat([df1, df2], ignore_index=True)
df['Date'] =  pd.to_datetime(df['Date'], format='mixed')
df['Full Text'] = df['Full Text'].astype(str)
df = df.sort_values(by='Date', ascending=True)
df.head(5)

<ipython-input-5-2856186f2e73>:1: DtypeWarning: Columns (17,42,61,75) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('russia_after_48hrs - Correct.csv', encoding = 'utf8')


,Query Id,Query Name,Date,Url,Domain,Sentiment,Country Code,Continent Code,Continent,Country,...,Parent Post Id,Pub Type,Publisher Sub Type,Rating,Region,Region Code,Station Name,Subscriptions,Emotion,"ï»¿""Query Id"""
83875,NaN,Putin Interview,2024-02-06 23:00:00,http://twitter.com/EloiHerby/statuses/17550034...,twitter.com,neutral,USA,NORTH AMERICA,North America,United States of America,...,NaN,Twitstream,NaN,NaN,Florida,USA.FL,NaN,NaN,NaN,2.002001e+09
83874,NaN,Putin Interview,2024-02-06 23:00:01,http://twitter.com/PeterWysocki/statuses/17550...,twitter.com,negative,USA,NORTH AMERICA,North America,United States of America,...,NaN,Twitstream,NaN,NaN,Alabama,USA.AL,NaN,NaN,NaN,2.002001e+09
83873,NaN,Putin Interview,2024-02-06 23:00:01,http://twitter.com/eddieoh/statuses/1755003441...,twitter.com,negative,USA,NORTH AMERICA,North America,United States of America,...,NaN,Twitstream,NaN,NaN,Minnesota,USA.MN,NaN,NaN,NaN,2.002001e+09
83872,NaN,Putin Interview,2024-02-06 23:00:03,http://twitter.com/JerryWestgate/statuses/1755...,twitter.com,neutral,USA,NORTH AMERICA,North America,United States of America,...,NaN,Twitstream,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.002001e+09
83871,NaN,Putin Interview,2024-02-06 23:00:06,http://twitter.com/QuickDrawHamltn/statuses/17...,twitter.com,negative,USA,NORTH AMERICA,North America,United States of America,...,NaN,Twitstream,NaN,NaN,New York,USA.NY,NaN,NaN,Fear,2.002001e+09


In [ ]:
for i in df.columns:
    print(i)

Query Id
Query Name
Date
Url
Domain
Sentiment
Country Code
Continent Code
Continent
Country
City Code
Account Type
Added
Assignment
Author
Avatar
Category Details
Checked
City
Display URLs
Engagement Score
Entity Info
Expanded URLs
Full Name
Full Text
Gender
Hashtags
Impact
Impressions
Interest
Last Assignment Date
Latitude
Location Name
Longitude
Media Filter
Media URLs
Mentioned Authors
Original Url
Priority
Professions
Resource Id
Short URLs
Starred
Status
Subtype
Thread Author
Thread Created Date
Thread Entry Type
Thread Id
Thread URL
Total Monthly Visitors
X Author ID
X Channel Role
X Followers
X Following
X Replies
X Reply to
X Repost of
X Reposts
X Likes
X Posts
X Verified
Updated
Reach (new)
Ad Value
Air Type
Author Verified Type
Blog Name
Broadcast Media Url
Broadcast Type
Content Source
Content Source Name
Copyright
Engagement Type
Factiva Attribute Code
Is Syndicated
Item Review
Media Type
Mention Id
Parent Blog Name
Parent Post Id
Pub Type
Publisher Sub Type
Rating
Region
R

In [ ]:
# non_df = df[~df['Snippet'].str.startswith('RT @')] # remove retweets
# original_df = df[df['Engagement Type'].isna() | (df['Engagement Type'] == '')] #only keep the original tweets

def process_tweet(tweet):
    """Process the tweet text to remove the RT @username: prefix."""
    if isinstance(tweet, str) and tweet.startswith('RT @'):
        parts = tweet.split(' ', 2)
        if len(parts) > 2:
            tweet = parts[2].lstrip()
    tweet = re.sub(r'@\w+', '', tweet)
    tweet = re.sub(r'http[s]?://\S+', '', tweet)
    tweet = tweet.strip()
    return tweet

df['original_text'] = df['Full Text'].apply(process_tweet)

retweet_df = df[df['Engagement Type'] == 'RETWEET']
retweet_df = retweet_df.drop_duplicates(subset='original_text', keep='first') # drop duplicated retweets based on retweets
nonretweet_df = df[df['Engagement Type'] != 'RETWEET']

df1 = pd.concat([retweet_df, nonretweet_df])
df1 = df1[~((df1.duplicated(subset='original_text', keep=False)) & (df1['Engagement Type'] == 'RETWEET'))] # drop duplicated retweets based on original tweets
df1 = df1.sort_index()
df1 = df1.reset_index(drop=True)

df1 = df1.drop_duplicates(subset="original_text")

print(f"The number of all tweets: {len(df)}")
print(f"The number of original tweets: {len(df1)}")

The number of all tweets: 146993
The number of original tweets: 51948


In [ ]:
docs = df1['original_text'].to_list()

### BERTopic modeling

In [ ]:
# if AttributeError: module 'openai' has no attribute 'OpenAI', run the following in the terminal
# conda create -n bertopic_env python=3.10
# conda activate bertopoic_env
# pip install bertopic
!pip install bertopic
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.cluster import KMeans
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import KeyBERTInspired

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.9 MB/s eta 0:00:00


### Topic Modeling - KMeans with MiniLM embedding

In [ ]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
umap_model = UMAP(n_neighbors=200, n_components=5, min_dist=0.0, metric='cosine') #default n_neighbors=15
vectorizer_model = CountVectorizer()
cluster_model = KMeans(n_clusters=50) #adjust the number of clusters based on the plot above
ctfidf_model = ClassTfidfTransformer()
representation_model = KeyBERTInspired()

topic_model = BERTopic(
 embedding_model=embedding_model,          # Step 1 - Extract embeddings
 umap_model=umap_model,                    # Step 2 - Reduce dimensionality
 hdbscan_model=cluster_model,              # Step 3 - Cluster reduced embeddings
 vectorizer_model=vectorizer_model,        # Step 4 - Tokenize topics
 ctfidf_model=ctfidf_model,                # Step 5 - Extract topic words
 representation_model=representation_model # Step 6 - (Optional) Fine-tune topic represenations
                     # Increase number of representative documents per topic
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
topics, probs = topic_model.fit_transform(docs)
# Retrieve topics
topics = topic_model.get_topics()

In [ ]:
# Prepare your documents to be used in a dataframe
documents = pd.DataFrame({"Document": docs,
                          "ID": range(len(docs)),
                          "Topic": topic_model.topics_})

# Extract the top 10 representative documents
repr_docs = topic_model._extract_representative_docs(c_tf_idf=topic_model.c_tf_idf_,
                                                     documents=documents,
                                                     topics=topic_model.topic_representations_,
                                                     nr_repr_docs=10)

# Check if repr_docs is a tuple and unpack it
if isinstance(repr_docs, tuple):
    repr_docs = repr_docs[0]  # Assuming the first element is the desired dictionary

flattened_data = [(topic, doc) for topic, docs in repr_docs.items() for doc in docs]
df_repr_docs = pd.DataFrame(flattened_data, columns=["Topic", "Representative_Doc"])
df_repr_docs
df_repr_docs.to_excel("MiniLM Representative Docs - 10.xlsx", index = False)

In [ ]:
topic_model.visualize_barchart()

In [ ]:
topic_model.visualize_topics()

### Topic Modeling with HuggingFace distilbert embedding

In [ ]:
from transformers.pipelines import pipeline
import torch

device = 0 if torch.cuda.is_available() else -1

embedding_model = pipeline("feature-extraction", model="distilbert-base-cased", device=device)
umap_model = UMAP(n_neighbors=200, n_components=5, min_dist=0.0, metric='cosine')
vectorizer_model = CountVectorizer()
cluster_model = KMeans(n_clusters=50) #adjust the number of clusters based on the plot above
ctfidf_model = ClassTfidfTransformer()
representation_model = KeyBERTInspired()

topic_model = BERTopic(
  embedding_model=embedding_model,          # Step 1 - Extract embeddings
  umap_model=umap_model,                    # Step 2 - Reduce dimensionality
  hdbscan_model=cluster_model,              # Step 3 - Cluster reduced embeddings
  vectorizer_model=vectorizer_model,        # Step 4 - Tokenize topics
  ctfidf_model=ctfidf_model,                # Step 5 - Extract topic words
  representation_model=representation_model # Step 6 - (Optional) Fine-tune topic represenations
)

topics, probs = topic_model.fit_transform(docs)

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
representation = topic_model.get_topic_info()
representation.to_excel("kmeans_distilbert_50_topic_representation.xlsx", index=False)
document_info = topic_model.get_document_info(docs)
document_info.to_excel("kmeans_distilbert_50_document_representation.xlsx", index=False)
representation

,Topic,Count,Name,Representation,Representative_Docs
0,0,2138,0_russia_ukraine_america_putin,"[russia, ukraine, america, putin, think, shoul...",[But we will soon see for certain who wants pe...
1,1,1814,1_tucker_putin_russia_think,"[tucker, putin, russia, think, fucking, ukrain...","[Why do you believe Putin?, Why don't you move..."
2,2,1745,2_russia_russian_ukrainian_ukraine,"[russia, russian, ukrainian, ukraine, poland, ...",[The new talking point is that NATO states mus...
3,3,1736,3_illegals_israel_billions_ukraine,"[illegals, israel, billions, ukraine, dems, re...","[It’s an Ukraine funding bill, not a border bi..."
4,4,1724,4_knows_idiot_tucker_got,"[knows, idiot, tucker, got, looks, putin, fuck...","[You should move to Russia!, Putin is just ano..."
5,5,1717,5_soviet_russia_russians_russian,"[soviet, russia, russians, russian, nazis, uss...",[So all Putin has to say to justify an invasio...
6,6,1688,6_russia_ukraine_america_putin,"[russia, ukraine, america, putin, american, sa...",[We didn't have missiles or military bases in ...
7,7,1678,7_tucker_russia_really_putin,"[tucker, russia, really, putin, thing, wouldn,...","[Or, maybe trying to make up for the things we..."
8,8,1672,8_tucker_russia_russian_putin,"[tucker, russia, russian, putin, biden, unders...",[I thought it was an amazing interview. Putin ...
9,9,1659,9_tucker_really_putin_watched,"[tucker, really, putin, watched, watching, bid...",[Do you think they caught him out of guard? Si...


In [ ]:
topic_model.visualize_barchart()

In [ ]:
topic_model.visualize_topics()